In [1]:
import pandas as pd 
import numpy as np 
import os
import glob

In [2]:
excel_names = glob.glob("/Users/taylorphillips/galvanize/capstone/test_files/*.xlsx")

In [3]:
excel_names

['/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900035_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900001_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900038_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900045_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900008_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900037_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900044_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900039_Production_11-Feb-2019.xlsx',
 '/Users/taylorphillips/galvanize/capstone/test_files/API_04_02900036_Production_11-Feb-2019.xlsx']

In [4]:
excels = [pd.ExcelFile(name) for name in excel_names]

In [5]:
excels

In [6]:
frames = [x.parse(x.sheet_names[0], header=3) for x in excels]

In [7]:
type(frames[0])

pandas.core.frame.DataFrame

In [8]:
frames[1:] = [df[1:] for df in frames[1:]]

In [9]:
combined = pd.concat(frames)
combined

,API Number,Production Date,Oil Produced (bbl),Water Produced (bbl),Gas Produced (Mcf),Days Well Produced,Gravity of Oil,Casing Pressure,Tubing Pressure,BTU,Method of Operation,Water Disposition,PWT Status,Well Type,Status,Pool Code,Reported Date
0,2900035,2017 Total,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
1,2900035,Sep-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-09-01
2,2900035,Aug-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-08-01
3,2900035,Jul-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-07-01
4,2900035,Jun-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-06-01
5,2900035,May-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-05-01
6,2900035,Feb-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-02-01
7,2900035,Jan-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-01-01
8,2900035,2016 Total,0.0,0.0,21.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-01
9,2900035,Dec-2016,0.0,0.0,15.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,A,OG,NaN,0.0,2016-12-01


In [10]:
type(combined)

pandas.core.frame.DataFrame

In [11]:
combined.columns = combined.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
combined.columns

Index(['API_Number', 'Production_Date', 'Oil_Produced_bbl',
       'Water_Produced_bbl', 'Gas_Produced_Mcf', 'Days_Well_Produced',
       'Gravity_of_Oil', 'Casing_Pressure', 'Tubing_Pressure', 'BTU',
       'Method_of_Operation', 'Water_Disposition', 'PWT_Status', 'Well_Type',
       'Status', 'Pool_Code', 'Reported_Date'],
      dtype='object')

In [12]:
combined.columns = [x.lower() for x in combined.columns]
combined.columns

Index(['api_number', 'production_date', 'oil_produced_bbl',
       'water_produced_bbl', 'gas_produced_mcf', 'days_well_produced',
       'gravity_of_oil', 'casing_pressure', 'tubing_pressure', 'btu',
       'method_of_operation', 'water_disposition', 'pwt_status', 'well_type',
       'status', 'pool_code', 'reported_date'],
      dtype='object')

In [13]:
combined = combined[pd.notnull(combined['well_type'])]
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4022 entries, 1 to 533
Data columns (total 17 columns):
api_number             4022 non-null int64
production_date        4022 non-null object
oil_produced_bbl       3959 non-null float64
water_produced_bbl     3954 non-null float64
gas_produced_mcf       3950 non-null float64
days_well_produced     3940 non-null float64
gravity_of_oil         3891 non-null float64
casing_pressure        3602 non-null float64
tubing_pressure        3603 non-null float64
btu                    3006 non-null float64
method_of_operation    3877 non-null float64
water_disposition      3709 non-null float64
pwt_status             4022 non-null object
well_type              4022 non-null object
status                 3613 non-null float64
pool_code              4022 non-null float64
reported_date          4022 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(12), int64(1), object(3)
memory usage: 565.6+ KB


In [14]:
combined

,api_number,production_date,oil_produced_bbl,water_produced_bbl,gas_produced_mcf,days_well_produced,gravity_of_oil,casing_pressure,tubing_pressure,btu,method_of_operation,water_disposition,pwt_status,well_type,status,pool_code,reported_date
1,2900035,Sep-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-09-01
2,2900035,Aug-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-08-01
3,2900035,Jul-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-07-01
4,2900035,Jun-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-06-01
5,2900035,May-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-05-01
6,2900035,Feb-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-02-01
7,2900035,Jan-2017,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2017-01-01
9,2900035,Dec-2016,0.0,0.0,15.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,A,OG,NaN,0.0,2016-12-01
10,2900035,Nov-2016,0.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,NaN,0.0,2016-11-01
11,2900035,Oct-2016,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,A,OG,6.0,0.0,2016-10-01


In [ ]:
frames[0].sum()

In [ ]:
frames[1].sum()

In [18]:
summary_excel = glob.glob("/Users/taylorphillips/galvanize/capstone/*.xlsx")
summary_excel

['/Users/taylorphillips/galvanize/capstone/CA Well Search Results 12-Feb-2019.xlsx']

In [24]:


excel_names = glob.glob("/Users/taylorphillips/galvanize/capstone/*.xlsx")

excels = [pd.ExcelFile(name) for name in excel_names]

frames = [x.parse(x.sheet_names[0], header=3) for x in excels]

combined = pd.concat(frames)

combined.columns = combined.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

combined.columns = [x.lower() for x in combined.columns]

combined.to_csv("summary.csv", index=False)

In [22]:
sum_parse()

,district_#,formatted_api_#,operator_name,operator_code,field_name,field_code,api_#,lease_name,well_#,well_status,...,longitude,gissourcecode,datumcode,blmwell,dryhole,directional,hydraulically_fractured,spud_date,completion_date,abandoned_date
0,4,029-00001,Chevron U.S.A. Inc.,C5640,Kern River,340,2900001,NaN,12-11,A,...,-118.958760,gps,83,N,N,N,,11/30/2008,NaN,NaN
1,4,029-00008,"Vaquero Energy, Inc.",V0725,Edison,222,2900008,Walter Brown,6,I,...,-118.841938,gps,83,N,N,N,,NaN,NaN,NaN
2,4,029-00035,"Maranatha Petroleum, Inc.",M1600,Fruitvale,256,2900035,Pace Fee,9,A,...,-119.045723,gps,83,N,N,N,,10/04/1989,NaN,NaN
3,4,029-00036,Naftex Operating Company,N0097,Edison,222,2900036,Racetrack,6-5,A,...,-118.851914,gps,83,N,N,N,,NaN,NaN,NaN
4,4,029-00037,"Grayson Service, Inc.",G2810,Canal,104,2900037,KCL-E,23,A,...,-119.256466,gps,83,N,N,N,,NaN,NaN,NaN
5,4,029-00038,Chevron U.S.A. Inc.,C5640,Kern River,340,2900038,Canfield,502,A,...,-118.982517,opr,83,N,N,N,,NaN,NaN,NaN
6,4,029-00039,Chevron U.S.A. Inc.,C5640,Kern River,340,2900039,Canfield,57,A,...,-118.979495,opr,83,N,N,N,,NaN,NaN,NaN
7,4,029-00044,Chevron U.S.A. Inc.,C5640,Kern River,340,2900044,Kern,36,A,...,-118.985182,opr,83,N,N,N,,NaN,NaN,NaN
8,4,029-00045,Chevron U.S.A. Inc.,C5640,Kern River,340,2900045,Blinn,18,A,...,-118.985129,opr,83,N,N,N,,NaN,NaN,NaN
9,4,029-00047,Chevron U.S.A. Inc.,C5640,Kern River,340,2900047,San Joaquin,280,A,...,-118.991276,opr,83,N,N,N,,NaN,NaN,NaN


In [25]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77978 entries, 0 to 77977
Data columns (total 28 columns):
district_#                 77978 non-null int64
formatted_api_#            77978 non-null object
operator_name              77978 non-null object
operator_code              77978 non-null object
field_name                 77978 non-null object
field_code                 77978 non-null int64
api_#                      77978 non-null int64
lease_name                 51344 non-null object
well_#                     77978 non-null object
well_status                77978 non-null object
pool_welltypes             77978 non-null object
section                    77978 non-null int64
township                   77978 non-null object
range                      77978 non-null object
base_meridian              77978 non-null object
area_code                  77978 non-null int64
area_name                  77978 non-null object
latitude                   77978 non-null float64
longitude    